In [0]:
# parameteize the notebook to make it dynamic
dbutils.widgets.text("file_name", "")

In [0]:
file_name = dbutils.widgets.get("file_name")

In [0]:
# read data from source using auto-loader

df = (spark.readStream.format("cloudFiles")
        .option("cloudFiles.format", "parquet")
        .option(f"cloudFiles.schemaLocation", "abfss://bronze@databricksdevetl.dfs.core.windows.net/checkpoint_{file_name}")
        .load(f"abfss://source@databricksdevetl.dfs.core.windows.net/{file_name}")
)

In [0]:
# write data to bronze layer using auto-loader

df.writeStream.format("parquet")\
    .option("checkpointLocation", f"abfss://bronze@databricksdevetl.dfs.core.windows.net/checkpoint_{file_name}")\
    .option("path", f"abfss://bronze@databricksdevetl.dfs.core.windows.net/{file_name}")\
    .outputMode("append")\
    .trigger(once=True)\
    .start()

In [0]:
df = spark.read.format("parquet").load(f"abfss://bronze@databricksdevetl.dfs.core.windows.net/{file_name}")
df.display()